new script for non transformed data, gridsearch

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics.pairwise import chi2_kernel, rbf_kernel, linear_kernel, polynomial_kernel, sigmoid_kernel

In [2]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

In [3]:
X_columns = ['Open', 'High', 'Low', 'Volume']
X_values = df_train[X_columns].values
X_test_values=df_test[X_columns].values
y_values= df_train["Close"].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_values, y_values, test_size=0.06, random_state=0)

In [5]:
def getLinearRegressorConfig():
    model = LinearRegression(n_jobs=-1)
    parameters = {
        "positive": [True, False]
    }
    return model, parameters

def getRidgeRegressorConfig():
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
    model = Ridge(random_state=0, max_iter=None)
    parameters = {
        "alpha": np.arange(0.1, 3.1, 0.1),
        "positive": [True, False],
#         "solver": ["svd", "cholesky", ]
    }
    return model, parameters

# def getLassoRegressorConfig():
#     model = Lasso(random_state=0, max_iter=10000)
#     parameters = {
#         "alpha": np.arange(0.1, 3.1, 0.1),
# #         "precompute": ["auto", True, False],
#         "tol": [1e-4, 1e-3, 1e-2, 1e-1, 0, 1e1, 1e2, 1e3],
#         "positive": [True, False],
#         "selection": ["cyclic", "random"]
#     }
#     return model, parameters

# def getKernelRidgeRegressorConfig():
#     model = KernelRidge()
#     parameters = {
#         "alpha": np.arange(0.1, 3.1, 0.1),
#         "kernel": ["linear", rbf_kernel, polynomial_kernel, sigmoid_kernel, chi2_kernel],
#         "gamma": np.arange(0.1, 3.1, 0.1),
#         "degree": [3, 4, 5]
#     }
#     return model, parameters

# def getSVMRegressorConfig():
#     model = SVR(max_iter=10000)
#     parameters = {
#         "epsilon": np.arange(0.1, 3.1, 0.1),
#         "tol": [1e-4, 1e-3, 1e-2, 1e-1, 0, 1e1, 1e2, 1e3],
#         "C": np.arange(0.1, 3.1, 0.1),
#         "kernel":["rbf","poly","linear","sigmoid","precomputed"],
# 	"degree":[3,4,5]
#     }
#     return model, parameters

# def getKNeighboursRegressorConfig():
#     model = KNeighborsRegressor(n_jobs=-1, p=2)
#     parameters = {
#         "n_neighbors": [3, 4, 5, 6, 7, 8],
#         "weights": ["uniform", "distance"],
#         "algorithm": ["ball_tree", "kd_tree", "brute"]
#         # "leaf_size": [25, 30, 35, 40, 45, 50]
#     }
#     return model, parameters

In [6]:
functions = [name for (name, thing) in locals().items() if callable(thing)]
model_functions = [eval(function) for function in functions if function.endswith("RegressorConfig")]
model_functions

[<function __main__.getLinearRegressorConfig()>,
 <function __main__.getRidgeRegressorConfig()>]

In [7]:
for model_function in model_functions:
    try:
        model, parameters = model_function()
        search = GridSearchCV(model, parameters, n_jobs=-1, refit=True)
        search.fit(X_train, y_train)
        print(search.best_params_, search.best_score_)
    except Exception as error_message:
        print(f"Error in model {model_function}: {error_message}")

{'positive': False} 0.9997907160858851
{'alpha': 3.0000000000000004, 'positive': False} 0.9997907284712237


In [42]:

def getLinearRegressorConfigTrained():
    model=LinearRegression(positive=False)
    re
def getRidgeRegressorConfigTrained():
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
    model = Ridge(random_state=0, max_iter=None, alpha=3.0000000000000004, positive= False,solver="cholesky")
    return model

# def getLassoRegressorConfigTrained():
#     model = Lasso(random_state=0, max_iter=10000,alpha=0.1, positive=False, selection="cyclic",tol=0)
#     return model

# def getKernelRidgeRegressorConfigTrained():
#     model = KernelRidge(alpha=0.7000000000000001,degree=3, gamma=0.1, kernel="linear")
#     return model

# def getKNeighboursRegressorConfigTrained():
#     model = KNeighborsRegressor(n_jobs=-1, p=2,algorithm="ball_tree",n_neighbors=8, weights="uniform")
#     return model

In [32]:
functions = [name for (name, thing) in locals().items() if callable(thing)]
model_functions = [eval(function) for function in functions if function.endswith("RegressorConfigTrained")]

In [33]:
model_functions

[<function __main__.getRidgeRegressorConfigTrained()>]

In [34]:
for model in model_functions:
    model=model()
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test_values)
    df_submission=pd.read_csv("../sample_submission.csv")
    df_submission["Close"]=y_pred
    df_submission.to_csv(f"submission_{model}.csv",index=False)